In [2]:
# with all production modes
# make corrections to signal yield for the simulation modelling uncertainties and create datacard

import ROOT as rt
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard, make_datacard_2sig, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [60]:
fpath =OrderedDict()
tree = OrderedDict()

prod = ['ggH']
old_ctau = {
    'STodd_ms3p0':[100,500],
    'SToEE_ms0p4':[10,50],
    'SToEE_ms1p0':[25,125],
    'SToEE_ms2p0':[50,250],
    'SToEE_ms4p0':[100,500],
    'SToGammaGamma_ms0p4':[10,50],
    'SToGammaGamma_ms1p0':[25,125],
    'SToGammaGamma_ms2p0':[50,250],
    'SToGammaGamma_ms4p0':[100,500],
    'SToGlueGlue_ms3p0':[100,500],
#     'SToKPlusKMinus_ms1p5':[38,187],
#     'SToK0K0_ms1p5':[38,187],
    'SToPi0Pi0_ms0p4':[10,50],
    'SToPi0Pi0_ms1p0':[25,125],
    'SToPiPlusPiMinus_ms0p4':[10,50],
    'SToPiPlusPiMinus_ms1p0':[25,125],
    'SToPiPlusPiMinus_ms2p0':[50,250],
    'SToPiPlusPiMinus_ms4p0':[100,500],
}
ntupler_version = 'V1p17/'
analyzer_version  = '/v2/vtest/'
mc_path = {}
mc_path['ggH']= '/storage/cms/store/user/jschindl/displacedJetMuonAnalyzer/low_mass_csc/V1p17/MC_Fall18/v2/v8/normalized/'

for p in prod:
    for k in old_ctau.keys():
        for ct in old_ctau[k]:
            key = 'MC_'+p+'_'+k + '_'+str(ct)
            fpath[key] = mc_path[p]+p+'_HToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
            if k == 'SToEE_ms4p0':
                fpath[key] ='/storage/cms/store/user/jschindl/displacedJetMuonAnalyzer/low_mass_csc/V1p17/MC_Fall18/v2/v9_no_met_cut/normalized/'+\
                p+'_HToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
            

data_path = '/storage/cms/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v100/normalized/'
fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'

NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    
    w = tree[k]["weight"].array()
    if not 'data' in k: 
        print(k, root_dir['NEvents']._fEntries)
        


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

MC_ggH_STodd_ms3p0_100 811000.0
MC_ggH_STodd_ms3p0_500 789700.0
MC_ggH_SToEE_ms0p4_10 979000.0
MC_ggH_SToEE_ms0p4_50 993400.0
MC_ggH_SToEE_ms1p0_25 989200.0
MC_ggH_SToEE_ms1p0_125 1252300.0
MC_ggH_SToEE_ms2p0_50 1007399.0
MC_ggH_SToEE_ms2p0_250 952000.0
MC_ggH_SToEE_ms4p0_100 1194100.0
MC_ggH_SToEE_ms4p0_500 942400.0
MC_ggH_SToGammaGamma_ms0p4_10 1124600.0
MC_ggH_SToGammaGamma_ms0p4_50 1195600.0
MC_ggH_SToGammaGamma_ms1p0_25 440000.0
MC_ggH_SToGammaGamma_ms1p0_125 950000.0
MC_ggH_SToGammaGamma_ms2p0_50 1203700.0
MC_ggH_SToGammaGamma_ms2p0_250 1193500.0
MC_ggH_SToGammaGamma_ms4p0_100 1039900.0
MC_ggH_SToGammaGamma_ms4p0_500 1190200.0
MC_ggH_SToGlueGlue_ms3p0_100 1219300.0
MC_ggH_SToGlueGlue_ms3p0_500 1208700.0
MC_ggH_SToPi0Pi0_ms0p4_10 957700.0
MC_ggH_SToPi0Pi0_ms0p4_50 898900.0
MC_ggH_SToPi0Pi0_ms1p0_25 1200300.0
MC_ggH_SToPi0Pi0_ms1p0_125 1225000.0
MC_ggH_SToPiPlusPiMinus_ms0p4_10 818000.0
MC_ggH_SToPiPlusPiMinus_ms0p4_50 840100.0
MC_ggH_SToPiPlusPiMinus_ms1p0_25 760300.0
MC_ggH_SToPi

In [57]:
NStation_ratio = [0.8153731, 1.135363]
AvgStation_ratio = [0.5585293, 1.070362, 1.320824, 0.9593284]
Eta_ratio = [1.0, 0.2841959,0.5748884,0.6125545,1.038548,1.184251,1.352579,0.5508717,0.82808,1.382264,1.030224,1.223585]


# nCsc with different hit vetoing

In [61]:

JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50
cut_based = True
cut_based_version = 'v4'

intime = True
DPHI_CUT = 1
weight = {}

nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
dphiMet_cluster = {}

cscRechitClusterEta = {}
cscRechitClusterNStation10  = {}
cscRechitClusterAvgStation10 = {}

sel_ev = {}
bdt_sel = {}


ggZH_weight = {}
higgsEta = {}
higgsPt = {}


cluster_index = '3'

NStation_weight = {}
AvgStation_weight = {}
Eta_weight = {}


for k in list(tree.keys())+['data_intime_sr']:
########### SELECTION: CLUSTERS ############
    if 'data' in k: T = tree['data']
    else: T = tree[k]

    sel_rechitcluster = np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')) < 2.0
    
    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12') <= me1112_veto)
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_MB1Seg_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RE12_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RB1_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpread') <= 20)

    if 'oot' in k:
        
        sel_rechitcluster = np.logical_and(sel_rechitcluster,  T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < -12.5)
    else:
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5.0))


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < JET_PT_CUT)
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)


########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)

########### SELECTION: EVENTS ############

    sel_ev[k] = T.array('METNoMuTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') >= 200)
    sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('nLeptons') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() >= 1)
    sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
    
########### BRANCHES ############

    
    ##### bdt variables ####

    cscRechitClusterNStation10[k] = T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterAvgStation10[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    
    if len(cscRechitClusterAvgStation10[k])>0:
        if cut_based:     
            if cut_based_version == 'v4':
                cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==2, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==3, np.abs(cscRechitClusterEta[k]) < 1.6)
                cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==4, np.abs(cscRechitClusterEta[k]) < 1.8)
                cond1 = np.logical_and(cscRechitClusterNStation10[k]==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation10[k])==1, cond2), np.logical_or(cond3, cond4)))
                cond2 = np.logical_and(cscRechitClusterNStation10[k] > 1, np.abs(cscRechitClusterEta[k]) < 1.9)
                bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))
            else:
                print("CUT BASED ERROR")
            if 'vr' in k:
                bdt_sel[k] = np.logical_not(bdt_sel[k])
        else:
            print('bdt based')
            if k == 'data_oot_sr':
                bdt_sel[k] = bdt_score[k] >= BDT_CUT
            elif 'vr' in k:
                bdt_sel[k] = bdt_score[k] < BDT_CUT            
            elif k == 'data':
                bdt_sel[k] = bdt_score[k] < BDT_CUT 
            else:
                bdt_sel[k] = bdt_score[k] >= BDT_CUT
        print("effiency",np.count_nonzero(bdt_sel[k])/len(bdt_sel[k]))

        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]



        higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][bdt_sel[k]]
        higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][bdt_sel[k]]


        
        ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]


        if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][bdt_sel[k]]
        else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][bdt_sel[k]]
        if 'ggZH' in k: weight[k] *= ggZH_weight[k]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]][:,0]
       
        

       
        NStation_weight[k] = np.ones(cscRechitClusterNStation10[k].shape)*NStation_ratio[0]
        NStation_weight[k][cscRechitClusterNStation10[k]>1] = NStation_ratio[1]
        
        AvgStation_weight[k] = np.ones(cscRechitClusterAvgStation10[k].shape)*AvgStation_ratio[0]
        AvgStation_weight[k][np.abs(cscRechitClusterAvgStation10[k])==2] = AvgStation_ratio[1]
        AvgStation_weight[k][np.abs(cscRechitClusterAvgStation10[k])==3] = AvgStation_ratio[2]
        AvgStation_weight[k][np.abs(cscRechitClusterAvgStation10[k])==4] = AvgStation_ratio[3]
        AvgStation_weight[k][cscRechitClusterNStation10[k]>1] = 1


        Eta_weight[k] = np.ones(cscRechitClusterEta[k].shape)*Eta_ratio[0]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>0.8, np.abs(cscRechitClusterEta[k])>0.9)] = Eta_ratio[0]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>0.9, np.abs(cscRechitClusterEta[k])>1.0)] = Eta_ratio[1]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.0, np.abs(cscRechitClusterEta[k])>1.1)] = Eta_ratio[2]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.1, np.abs(cscRechitClusterEta[k])>1.2)] = Eta_ratio[3]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.2, np.abs(cscRechitClusterEta[k])>1.3)] = Eta_ratio[4]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.3, np.abs(cscRechitClusterEta[k])>1.4)] = Eta_ratio[5]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.4, np.abs(cscRechitClusterEta[k])>1.5)] = Eta_ratio[6]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.5, np.abs(cscRechitClusterEta[k])>1.6)] = Eta_ratio[7]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.6, np.abs(cscRechitClusterEta[k])>1.7)] = Eta_ratio[8]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.7, np.abs(cscRechitClusterEta[k])>1.8)] = Eta_ratio[9]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.8, np.abs(cscRechitClusterEta[k])>1.9)] = Eta_ratio[10]
        Eta_weight[k][np.logical_and(np.abs(cscRechitClusterEta[k])>1.9, np.abs(cscRechitClusterEta[k])>2.0)] = Eta_ratio[11]


        
        if 'data' in k:
            NStation_weight[k] = NStation_weight[k]*0.0+1
            AvgStation_weight[k] = NStation_weight[k]*0.0 +1
    


        
    else:
        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        if 'ggH' in k:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight'))[sel_ev[k]]
        else:weight[k] = (T.array('weight')*T.array('pileupWeight'))[sel_ev[k]]
        if 'data' in k:weight[k] = np.ones(weight[k].shape, dtype=bool)
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]][:,0]
      
        print("no BDT applied")

effiency 0.884514435695538
effiency 0.890625
effiency 0.821917808219178
effiency 0.8803418803418803
effiency 0.8385650224215246
effiency 0.8481481481481481
effiency 0.8195121951219512
effiency 0.8036529680365296
effiency 0.8475177304964538
effiency 0.905829596412556
effiency 0.8181818181818182
effiency 0.8745519713261649
effiency 0.84375
effiency 0.8543689320388349
effiency 0.8458498023715415
effiency 0.8358778625954199
effiency 0.8189655172413793
effiency 0.8844765342960289
effiency 0.9024390243902439
effiency 0.908235294117647
effiency 0.8295964125560538
effiency 0.8440366972477065
effiency 0.8513513513513513
effiency 0.9003831417624522
effiency 0.8624161073825504
effiency 0.922360248447205
effiency 0.903010033444816
effiency 0.9098360655737705
effiency 0.8908296943231441
effiency 0.8685567010309279
effiency 0.8888888888888888
effiency 0.9010989010989011
effiency 0.2985971943887776
effiency 0.2985971943887776


# signal yield summary table

In [62]:
ctaus = [ '10', '50','100','500' ]


BR = 0.01
N_RECHIT_CUT = 130
DPHI_CUT = 0.75
corrections = 0.8953452999999999
print(ctaus)
var = dphiMet_cluster
for N_RECHIT_CUT in np.arange(60,220,10):
    if not N_RECHIT_CUT==130:continue
    total_sig = 0
    for m, OLD_CTAU in old_ctau.items():
        signal_rate = []
        unc_rate = []
        signal_unc = []
        
        
         
                            
        for ct in ctaus:   
#             if not ct == '1000':continue
            signal = 0
            sig_unc = 0
            shape_unc_temp = 0
            ctf = int(ct)
            if ctf < OLD_CTAU[0]:
                old_ctau_temp = np.array([OLD_CTAU[0]])
            else:
                for j, ct0 in enumerate(OLD_CTAU):
                    if ct0 == ctf: 
                        old_ctau_temp = np.array([int(ctf)])
                        break

                    elif ct0 > ctf:
                        old_ctau_temp = np.array([OLD_CTAU[j-1], OLD_CTAU[j]])
                        old_ctau_temp = np.array([OLD_CTAU[j]])
                        break
                    if j == len(OLD_CTAU)-1: 
                        old_ctau_temp = np.array([OLD_CTAU[j]])

            weight_sum = 0
            weight_len = 0
            for j,ct0 in enumerate(old_ctau_temp):
                prods = ['ggH',]
                for p in prods:
                    
                    k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)+''
                    

                    T = tree[k]
                    if np.count_nonzero(sel_ev[k]) == 0: continue
                    gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                    if len(gLLP_ctau) == 0: continue

                    w = weight[k]*AvgStation_weight[k][bdt_sel[k]]*NStation_weight[k][bdt_sel[k]]
                    w = weight[k]
                    # higgs pt, signal uncertainty weight
                    cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, np.abs(var[k])<DPHI_CUT)#bin D

                    signal += np.sum(w[cond])
#                     signal += np.count_nonzero(cond)
#                     print(np.mean(w[cond]))
                    sig_unc +=np.sum(w[cond]*w[cond])
#                     shape_unc_temp = (shape_unc_temp**2+np.sum(w[cond]*shape_unc[cond])**2)**0.5


            signal_rate.append(signal)
            signal_unc.append(sig_unc**0.5)
            unc_rate.append(shape_unc_temp)
        signal_rate = np.array(signal_rate)
        unc_rate = np.array(unc_rate)
        signal_unc = np.array(signal_unc)
#         print(N_RECHIT_CUT, '\t', signal_rate[0]*BR)
#         print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR*corrections])),'\\\\')
#         print(m,'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*limit[m]])),'\\\\')

#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_unc*BR])))
        print(m, '\t', '\t'.join(map(str, [round(num,5) for num in signal_rate*BR*corrections])))
#         print(m, '\t', '\t'.join(map(str, [round(num,1) for num in signal_rate*BR])))


#         print(m, '\t', '\t'.join(map(str, [round(num,2) for num in signal_unc*BR])))


#         print(m, '\t'.join(map(str,[round(num,2) for num in unc_rate*BR]))) #signal mc relative uncertainty
        total_sig += np.sum(signal_rate)
#         print(np.sum(signal_rate*BR))
# print(total_sig*BR)

#     print(m, '\t'.join(map(str,unc_rate/signal_rate))) #signal mc relative uncertainty



['10', '50', '100', '500']
STodd_ms3p0 	 20.59558	20.59558	20.59558	12.10852
SToEE_ms0p4 	 8.39103	7.92384	7.92384	7.92384
SToEE_ms1p0 	 8.74009	7.81862	7.81862	7.81862
SToEE_ms2p0 	 7.13373	7.13373	6.9144	6.9144
SToEE_ms4p0 	 8.36606	8.36606	8.36606	7.91653
SToGammaGamma_ms0p4 	 7.87693	7.84342	7.84342	7.84342
SToGammaGamma_ms1p0 	 9.45542	7.20815	7.20815	7.20815
SToGammaGamma_ms2p0 	 8.27502	8.27502	7.22132	7.22132
SToGammaGamma_ms4p0 	 8.63222	8.63222	8.63222	7.96635
SToGlueGlue_ms3p0 	 18.87252	18.87252	18.87252	11.32463
SToPi0Pi0_ms0p4 	 8.95742	6.74395	6.74395	6.74395
SToPi0Pi0_ms1p0 	 7.68351	7.31903	7.31903	7.31903
SToPiPlusPiMinus_ms0p4 	 16.74569	13.07742	13.07742	13.07742
SToPiPlusPiMinus_ms1p0 	 18.88823	15.52599	15.52599	15.52599
SToPiPlusPiMinus_ms2p0 	 18.36759	18.36759	14.35132	14.35132
SToPiPlusPiMinus_ms4p0 	 18.29623	18.29623	18.29623	14.91973


# signal uncertainties

In [63]:
jetVeto = {}
muonVeto = {}
rechitVeto = {}
cut_based_eff_unc = {}
clusterEff_unc = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
scaling = {}
lumi = {}
time_spread = {}
time = {}
theory = {}
pdf = {}
readout = {}
ggH_higgsPt , VBFH_higgsPt , WH_higgsPt , ZH_higgsPt , ttH_higgsPt , ggZH_higgsPt =({} for i in range(6)) 
ggH_qcdScale , VBFH_qcdScale , WH_qcdScale , ZH_qcdScale , ttH_qcdScale , ggZH_qcdScale =({} for i in range(6)) 
ggH_pdf , VBFH_pdf , WH_pdf , ZH_pdf , ttH_pdf , ggZH_pdf =({} for i in range(6)) 
Nrechit_weight_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/Uncertainties/Nrechit_weight.txt'

cut_based_unc = [0.0921, 0.0852] # muon Eta reweighted
clustering_unc = [0.1506, 0.1047]

for m in old_ctau.keys():
    for ct in old_ctau[m]:
        for p in prod:
            if p == 'others':k = 'MC_ggH_'+str(m)+'_'+str(ct)   
            else:k = 'MC_'+p+'_'+str(m)+'_'+str(ct)         
            if p == 'others': key = 'MC_others_'+str(m)+'_'+str(ct)   
            else: key = k
            print(key, k)

            lumi[key] = [0.018]*4
            jetVeto[key] = [0.00068]*4
            muonVeto[key] = [0.045]*4
            rechitVeto[key] = [0.001]*4
            time[key] = [0.009]*4
            time_spread[key] = [0.028]*4
            clusterEff_unc[key]  = [0.035] * 4 #shifting Nrechit cut from 130 to 135
            cut_based_eff_unc[key]=[0.051]*4 #muon eta reweighted
            readout[key]=[0.01]*4 
            JES[key] = [0.084,0.083,0.042,0.041]
            pileup[key] = [0.008, 0.0110, 0.0110, 0.0110]
            

            ggH_higgsPt[key], VBFH_higgsPt[key], WH_higgsPt[key] , ZH_higgsPt[key] , ttH_higgsPt[key] , ggZH_higgsPt[key] = ([0.0]*8,)*6
            ggH_qcdScale[key], VBFH_qcdScale[key], WH_qcdScale[key] , ZH_qcdScale[key] , ttH_qcdScale[key] , ggZH_qcdScale[key] = ([0.0]*8,)*6
            ggH_pdf[key] , VBFH_pdf[key] , WH_pdf[key] , ZH_pdf[key] , ttH_pdf[key] , ggZH_pdf[key] = ([0.0]*4,)*6
            if 'ggH' in key: 
                ggH_higgsPt[key] = [0.205, 0.205,0.207,0.208, 0.133,0.133,0.134,0.134] #down*4/up*4
                ggH_qcdScale[key] = [0.067]*4+[0.046]*4 #down/up
                ggH_pdf[key] = [0.032]*4

            elif 'VBFH' in key: 
                VBFH_higgsPt[key] = [0.046, 0.159, 0.007, 0.006, 0.022, 0.032, 0.012, 0.010] #down/up
                VBFH_qcdScale[key] = [0.003]*4+[0.004]*4 #down/up
                VBFH_pdf[key] = [0.021]*4
            elif 'ggZH' in key:
                ggZH_higgsPt[key] = [0.194, 0.198, 0.208, 0.208, 0.120, 0.123, 0.134, 0.133] #down/up,ggZH
                ggZH_qcdScale[key] = [0.251]*4+[0.189]*4 #down/up
                ggZH_pdf[key] = [0.024]*4
#                 scaling[key] = [0.2]*4
            elif 'ZH' in key: 
                ZH_higgsPt[key] = [0.125, 0.057, 0.01, 0.017, 0.033,0.018, 0.006, 0.006] #down/up
                ZH_qcdScale[key] = [0.006]*4+[0.005]*4 #down/up
                ZH_pdf[key] = [0.019]*4
            elif 'WH' in key: 
                WH_higgsPt[key] = [0.071,0.040, 0.003, 0.054, 0.026, 0.024, 0.004, 0.018] #down/up
                WH_qcdScale[key] = [0.007]*4+[0.005]*4 #down/up
                WH_pdf[key] = [0.019]*4
            elif 'ttH' in key: 
                ttH_higgsPt[key] = [0.012, 0.148, 0.051, 0.028, 0.007, 0.066, 0.016, 0.010] #down/up
                ttH_qcdScale[key] = [0.092]*4+[0.058]*4 #down/up
                ttH_pdf[key] = [0.036]*4
            mc_stats[k] = []
            if p == 'ggZH': scaling[key] = [0.2]*4
            else: scaling[key] = [0,0,0,0]


sig_eff = [lumi, muonVeto, jetVeto,rechitVeto,time, time_spread, cut_based_eff_unc, clusterEff_unc,readout, JES,pileup, \
           ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
           ggH_qcdScale, VBFH_qcdScale, WH_qcdScale, ZH_qcdScale, ttH_qcdScale, ggZH_qcdScale, \
           ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
           scaling, mc_stats]
sig_eff_name = ['lumi','muonVeto','jetVeto','rechitVeto','time','time_spread','cut_based_eff_unc', 'clusterEff_unc', 'readout', 'JES','pileup', \
                'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
                'ggH_qcdScale', 'VBFH_qcdScale', 'WH_qcdScale', 'ZH_qcdScale', 'ttH_qcdScale', 'ggZH_qcdScale', \
                'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
                'ggZH_reweight','mc_stats']


assert(len(sig_eff)==len(sig_eff_name))

MC_ggH_STodd_ms3p0_100 MC_ggH_STodd_ms3p0_100
MC_ggH_STodd_ms3p0_500 MC_ggH_STodd_ms3p0_500
MC_ggH_SToEE_ms0p4_10 MC_ggH_SToEE_ms0p4_10
MC_ggH_SToEE_ms0p4_50 MC_ggH_SToEE_ms0p4_50
MC_ggH_SToEE_ms1p0_25 MC_ggH_SToEE_ms1p0_25
MC_ggH_SToEE_ms1p0_125 MC_ggH_SToEE_ms1p0_125
MC_ggH_SToEE_ms2p0_50 MC_ggH_SToEE_ms2p0_50
MC_ggH_SToEE_ms2p0_250 MC_ggH_SToEE_ms2p0_250
MC_ggH_SToEE_ms4p0_100 MC_ggH_SToEE_ms4p0_100
MC_ggH_SToEE_ms4p0_500 MC_ggH_SToEE_ms4p0_500
MC_ggH_SToGammaGamma_ms0p4_10 MC_ggH_SToGammaGamma_ms0p4_10
MC_ggH_SToGammaGamma_ms0p4_50 MC_ggH_SToGammaGamma_ms0p4_50
MC_ggH_SToGammaGamma_ms1p0_25 MC_ggH_SToGammaGamma_ms1p0_25
MC_ggH_SToGammaGamma_ms1p0_125 MC_ggH_SToGammaGamma_ms1p0_125
MC_ggH_SToGammaGamma_ms2p0_50 MC_ggH_SToGammaGamma_ms2p0_50
MC_ggH_SToGammaGamma_ms2p0_250 MC_ggH_SToGammaGamma_ms2p0_250
MC_ggH_SToGammaGamma_ms4p0_100 MC_ggH_SToGammaGamma_ms4p0_100
MC_ggH_SToGammaGamma_ms4p0_500 MC_ggH_SToGammaGamma_ms4p0_500
MC_ggH_SToGlueGlue_ms3p0_100 MC_ggH_SToGlueGlue_ms3p0_100
MC

In [64]:
signal_correction = (1-0.015)*(1-0.033)*(1-0.060)
print(signal_correction)

0.8953452999999999


# make datacard

In [65]:
import os
import math
import importlib
importlib.reload(sys.modules['helper'])
from helper import make_datacard, make_datacard_2sig, weight_calc
#methodB use oot vr and ABCD method
#methodC use the fit functions
N_RECHIT_CUTS = np.arange(60,220, 10)
# N_RECHIT_CUTS = [130]

ctaus = ['1','3','5','10','15','20','30','40', '50','60', '100', '125','150','200','300','500','600','700','800','900','1000', '2000','3000','4000', '5000', '6000','7000','8000','10000', '20000','30000','50000',\
         '100000',]
UNBLIND = 2
# 0 use OOT region, 1 unblind ABC, 2 unblind ABCD
method = 2
datacard_version  = 'v13'
if datacard_version == 'v13': # no bkg uncertainty
    bkg_unc = [] #ABCD closure, sum of stats from two VR
    bkg_unc_name = []
else:
    bkg_unc = [] #list of two uncertainty, each element is N for the gamma uncertainty distriubtion, alpha is calculated in the makr_datacard method
    bkg_unc_name = ['ABCD_intimeVR', 'ABCD_OOTSR']
    print("ERROR in uncertainty assignment")

var = dphiMet_cluster
DPHI_CUT = 0.75
DPHI_CUTS = [0.75]

# print(bdtBkgEff_nrechit.shape)
bkg = []
print(N_RECHIT_CUTS)
for N_RECHIT_CUT in N_RECHIT_CUTS:
    if not N_RECHIT_CUT==130:continue
    for DPHI_CUT in DPHI_CUTS:
        bkg_unc = []
        ##### method B (use OOT high BDT region and tha ratio between intime and OOT)#####
       
        k = 'data_intime_sr'
        a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
        c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        bkg_rate = np.array([a,b,c,a*c/b])
        if UNBLIND == 1:observation= np.array([a,b,c,a*c/b])
        else: observation= np.array([a,b,c,d])
        print(N_RECHIT_CUT, observation, bkg_unc)
        
        #####
        # a = c*c1*c2
        # b = c1* c
        # c = c
        # d = c2*c
        # start adding signal
        #####
        outDataCardsDir = '/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/'+\
        ntupler_version+analyzer_version+'/'
        if cut_based:outDataCardsDir += 'cut_based_'+cut_based_version+'/'
        else: outDataCardsDir += bdt_name+'/'
        outDataCardsDir += datacard_version+'/methodB/'
        if UNBLIND == 0: outDataCardsDir += 'blind/'
        elif UNBLIND == 1: outDataCardsDir += 'unblindABC/'
        else:outDataCardsDir += 'unblind/'
        if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
        print(outDataCardsDir)
        sig_norm = []
        for m in old_ctau.keys():
            for ct in ctaus:
                dphi = str(DPHI_CUT).replace(".", "p")
                if len(prod) == 5: modelName = 'allProd_HToSS_'
                elif len(prod) ==6:  modelName = 'allProd_withggZH_HToSS_'
                else: modelName = ('_').join(prod)+'_HToSS_'
                modelName = modelName +str(m)+'_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiCluster'+dphi
                signal_rate = {}
                mc_stat_unc = {}
                gmn = {}
                sig_unc = {}
                OLD_CTAU = old_ctau[m].copy()
                if int(ct) < OLD_CTAU[0]: ct_list = [OLD_CTAU[0]]
                elif int(ct)>OLD_CTAU[-1]: ct_list = [OLD_CTAU[-1]]
                elif int(ct) == OLD_CTAU[0]: ct_list = [int(ct)]
                elif int(ct) == OLD_CTAU[1]: ct_list = [int(ct)]
                else:ct_list = OLD_CTAU.copy()
                for p in prod:                    
                    signal_rate[p] = np.zeros((4,))
                    mc_stat_unc[p] = np.zeros((4,))
                    gmn[p] = np.zeros((4,))
                    sig_unc[p] = []
                   
                    for i, ct0 in enumerate(ct_list):
                        k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)
                        T = tree[k]
                        if np.count_nonzero(sel_ev[k])==0:continue
#                             gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                        gLLP_ctau = T.array('gLLP_ctau')[sel_ev[k]][bdt_sel[k]]

                        weight_ctau = weight_calc(gLLP_ctau, int(ct)/10, int(ct0)/10) # convert everything to cm
                        gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                        if len(ct_list) == 1:weight_cond = gLLP_ctau >= 0
                        else:
                            if i == 0 : weight_cond = gLLP_ctau<int(ct_list[0]/2)
                            else: weight_cond = gLLP_ctau>=int(ct_list[0]/2)
                        w = weight[k]*weight_ctau
                        cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                        signal_rate[p][0]+=np.sum(w[cond])
                        mc_stat_unc[p][0]+=np.sum(w[cond]**2)
                        gmn[p][0] += len(w[cond])
                        cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT), weight_cond)
                        signal_rate[p][1]+=np.sum(w[cond])
                        mc_stat_unc[p][1]+=np.sum(w[cond]**2)
                        gmn[p][1] += len(w[cond])
                        cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                        signal_rate[p][2]+=np.sum(w[cond])
                        mc_stat_unc[p][2]+=np.sum(w[cond]**2)
                        gmn[p][2] += len(w[cond])
                        cond = np.logical_and(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT), weight_cond)
                        signal_rate[p][3]+=np.sum(w[cond])
                        mc_stat_unc[p][3]+=np.sum(w[cond]**2)
                        gmn[p][3] += len(w[cond])
                        # apply the corrections
                        for j in [0,1,2,3]:
                            if signal_rate[p][j]<0.0:
                                signal_rate[p][j]=0.0
                                mc_stat_unc[p][j]=0.0
                                gmn[p][j] = 0.0
                            else:
                                signal_rate[p][j] *= signal_correction

#                     mc_stat_unc[p] = list(np.nan_to_num(np.sqrt(mc_stat_unc[p])/signal_rate[p])) #convert absolute to relative uncertainty
                    
                    mc_stat_unc[p] = np.nan_to_num(signal_rate[p]/np.sqrt(mc_stat_unc[p]))**2 #gmn
                    mc_stat_unc[p][mc_stat_unc[p] == np.inf] = 0.0
                    
                    mc_stat_unc[p] = list(mc_stat_unc[p])
                    mc_stat_unc[p] = list(gmn[p])
                    if p == 'others': k = k.replace("ggH", "others")
                    for j, ele in enumerate(sig_eff):# go through each uncertainty
                        if j == len(sig_eff)-1: # if mc_stats
                            sig_unc[p].append(mc_stat_unc[p])
#                         elif sig_eff_name[j] in simulation_name:
# #                             print(type(ele[k]))
#                             sig_unc[p].append(np.array(ele[k])*0.5)
                        else:
                            sig_unc[p].append(ele[k])
                   
                
                norm = np.sum(signal_rate['ggH'])/4
                if norm == 0.0:continue
                sig_norm.append(norm)

                for k,v in signal_rate.items():signal_rate[k] = v/norm
                print(modelName, norm, signal_rate['ggH'], mc_stat_unc['ggH'])
                make_datacard_2sig(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                                  bkg_unc, bkg_unc_name, sig_unc, sig_eff_name)
                

[ 60  70  80  90 100 110 120 130 140 150 160 170 180 190 200 210]
130 [ 3 96 47  3] []
/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/V1p17//v2/vtest//cut_based_v4/v13/methodB/unblind/
ggH_HToSS_STodd_ms3p0_ctau1mm_nRechit130dPhiCluster0p75 6.505034439917522e-09 [0.00000000e+00 0.00000000e+00 9.56993565e-17 4.00000000e+00] [13.0, 5.0, 44.0, 275.0]
ggH_HToSS_STodd_ms3p0_ctau3mm_nRechit130dPhiCluster0p75 0.22770252036755878 [2.17080371e-14 0.00000000e+00 1.19874580e-05 3.99998801e+00] [13.0, 5.0, 44.0, 275.0]
ggH_HToSS_STodd_ms3p0_ctau5mm_nRechit130dPhiCluster0p75 5.570385886468398 [1.12072018e-08 7.01007002e-24 1.84731090e-03 3.99815268e+00] [13.0, 5.0, 44.0, 275.0]
ggH_HToSS_STodd_ms3p0_ctau10mm_nRechit130dPhiCluster0p75 51.244439289053744 [1.39526483e-04 7.56578369e-12 6.92002204e-02 3.93066025e+00] [13.0, 5.0, 44.0, 275.0]
ggH_HToSS_STodd_ms3p0_ctau15mm_nRechit130dPhiCluster0p75 116.83765427335531 [2.18176117e-03 5.281

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:130: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:130: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:130: RuntimeWarning: overflow encountered in square


ggH_HToSS_STodd_ms3p0_ctau1000mm_nRechit130dPhiCluster0p75 351.916340427079 [0.62872127 0.77029989 0.58490992 2.01606892] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau2000mm_nRechit130dPhiCluster0p75 169.01328414117734 [0.57207335 0.9391562  0.54433018 1.94444027] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau3000mm_nRechit130dPhiCluster0p75 96.4965070905857 [0.54538039 0.99458428 0.52796942 1.93206591] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau4000mm_nRechit130dPhiCluster0p75 61.91455567431059 [0.53081471 1.02135344 0.51937877 1.92845307] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau5000mm_nRechit130dPhiCluster0p75 42.96891179021091 [0.52173631 1.0370162  0.51411254 1.92713495] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau6000mm_nRechit130dPhiCluster0p75 31.52202216886434 [0.51555615 1.04727126 0.51055994 1.92661266] [47.0, 34.0, 43.0, 161.0]
ggH_HToSS_STodd_ms3p0_ctau7000mm_nRechit130dPhiCluster0p75 24.094383141111656 [0.51108343 1.05449857 0.5080